# GERANDO SESSÃO SPARK

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IcebergExample") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
    .getOrCreate()

25/10/02 15:04:22 WARN Utils: Your hostname, DESKTOP-JLUDO09 resolves to a loopback address: 127.0.1.1; using 172.27.76.95 instead (on interface eth0)
25/10/02 15:04:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/guizap/sparkiceberg-lab/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/guizap/.ivy2/cache
The jars for the packages stored in: /home/guizap/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9c5407dc-5f29-4182-8ced-39245d963b1e;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
:: resolution report :: resolve 135ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.sp

# LENDO CSV E TRATANDO OS DADOS DA COLUNA T PARA ANÁLISES

In [68]:
df = spark.read \
    .option("header", True) \
    .option("inferSchema", False) \
    .option("delimiter", ";") \
    .csv("data/sample.csv")

df = df.drop("_c15", "_c16")
df.printSchema()
df.show(5)

root
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- CO(GT): string (nullable = true)
 |-- PT08.S1(CO): string (nullable = true)
 |-- NMHC(GT): string (nullable = true)
 |-- C6H6(GT): string (nullable = true)
 |-- PT08.S2(NMHC): string (nullable = true)
 |-- NOx(GT): string (nullable = true)
 |-- PT08.S3(NOx): string (nullable = true)
 |-- NO2(GT): string (nullable = true)
 |-- PT08.S4(NO2): string (nullable = true)
 |-- PT08.S5(O3): string (nullable = true)
 |-- T: string (nullable = true)
 |-- RH: string (nullable = true)
 |-- AH: string (nullable = true)

+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+
|      Date|    Time|CO(GT)|PT08.S1(CO)|NMHC(GT)|C6H6(GT)|PT08.S2(NMHC)|NOx(GT)|PT08.S3(NOx)|NO2(GT)|PT08.S4(NO2)|PT08.S5(O3)|   T|  RH|    AH|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+----

In [70]:
from pyspark.sql.functions import regexp_replace, col

df = df.withColumn("T", regexp_replace(col("T"), r"[^0-9.]", "").cast("double"))
df = df.withColumn("T", col("T") / 10)

df.show(5)
df.printSchema()
df.select("T").show(20)

+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+
|      Date|    Time|CO(GT)|PT08.S1(CO)|NMHC(GT)|C6H6(GT)|PT08.S2(NMHC)|NOx(GT)|PT08.S3(NOx)|NO2(GT)|PT08.S4(NO2)|PT08.S5(O3)|   T|  RH|    AH|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+
|10/03/2004|18.00.00|   2,6|       1360|     150|    11,9|         1046|    166|        1056|    113|        1692|       1268|13.6|48,9|0,7578|
|10/03/2004|19.00.00|     2|       1292|     112|     9,4|          955|    103|        1174|     92|        1559|        972|13.3|47,7|0,7255|
|10/03/2004|20.00.00|   2,2|       1402|      88|     9,0|          939|    131|        1140|    114|        1555|       1074|11.9|54,0|0,7502|
|10/03/2004|21.00.00|   2,2|       1376|      80|     9,2|          948|    172|        1092|    122|        1584|       1203|11.0|60,0|

# CRIANDO A TABELA ICEBERG

In [71]:
df.write.format("iceberg").mode("overwrite").saveAsTable("local.sample_data")

In [72]:
spark.sql("SELECT * FROM local.sample_data").show()

+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+
|      Date|    Time|CO(GT)|PT08.S1(CO)|NMHC(GT)|C6H6(GT)|PT08.S2(NMHC)|NOx(GT)|PT08.S3(NOx)|NO2(GT)|PT08.S4(NO2)|PT08.S5(O3)|   T|  RH|    AH|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+
|10/03/2004|18.00.00|   2,6|       1360|     150|    11,9|         1046|    166|        1056|    113|        1692|       1268|13.6|48,9|0,7578|
|10/03/2004|19.00.00|     2|       1292|     112|     9,4|          955|    103|        1174|     92|        1559|        972|13.3|47,7|0,7255|
|10/03/2004|20.00.00|   2,2|       1402|      88|     9,0|          939|    131|        1140|    114|        1555|       1074|11.9|54,0|0,7502|
|10/03/2004|21.00.00|   2,2|       1376|      80|     9,2|          948|    172|        1092|    122|        1584|       1203|11.0|60,0|

In [73]:
spark.sql("SELECT * FROM local.sample_data WHERE T > 10").show()

+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+
|      Date|    Time|CO(GT)|PT08.S1(CO)|NMHC(GT)|C6H6(GT)|PT08.S2(NMHC)|NOx(GT)|PT08.S3(NOx)|NO2(GT)|PT08.S4(NO2)|PT08.S5(O3)|   T|  RH|    AH|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+
|10/03/2004|18.00.00|   2,6|       1360|     150|    11,9|         1046|    166|        1056|    113|        1692|       1268|13.6|48,9|0,7578|
|10/03/2004|19.00.00|     2|       1292|     112|     9,4|          955|    103|        1174|     92|        1559|        972|13.3|47,7|0,7255|
|10/03/2004|20.00.00|   2,2|       1402|      88|     9,0|          939|    131|        1140|    114|        1555|       1074|11.9|54,0|0,7502|
|10/03/2004|21.00.00|   2,2|       1376|      80|     9,2|          948|    172|        1092|    122|        1584|       1203|11.0|60,0|

# INSERT

In [83]:
spark.sql("""
INSERT INTO local.sample_data
VALUES (
    '15/03/2020',  -- Date
    '14.00.00',    -- Time
    '1.2',         -- CO(GT)
    '1300',        -- PT08.S1(CO)
    '50',          -- NMHC(GT)
    '0.3',         -- C6H6(GT)
    '1100',        -- PT08.S2(NMHC)
    '80',          -- NOx(GT)
    '1000',        -- PT08.S3(NOx)
    '40',          -- NO2(GT)
    '1500',        -- PT08.S4(NO2)
    '1200',        -- PT08.S5(O3)
    25.5,          -- T
    '45',          -- RH
    '0.8'          -- AH
)
""")


DataFrame[]

In [84]:
spark.sql("SELECT * from local.sample_data WHERE date = '15/03/2020'").show()

+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+---+---+
|      Date|    Time|CO(GT)|PT08.S1(CO)|NMHC(GT)|C6H6(GT)|PT08.S2(NMHC)|NOx(GT)|PT08.S3(NOx)|NO2(GT)|PT08.S4(NO2)|PT08.S5(O3)|   T| RH| AH|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+---+---+
|15/03/2020|14.00.00|   1.2|       1300|      50|     0.3|         1100|     80|        1000|     40|        1500|       1200|25.5| 45|0.8|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+---+---+



# UPDATE

In [85]:
spark.sql("""
UPDATE local.sample_data
SET T = T + 1.0
WHERE Date = '15/03/2020'
""")


DataFrame[]

In [86]:
spark.sql("SELECT * from local.sample_data WHERE date = '15/03/2020'").show()

+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+---+---+
|      Date|    Time|CO(GT)|PT08.S1(CO)|NMHC(GT)|C6H6(GT)|PT08.S2(NMHC)|NOx(GT)|PT08.S3(NOx)|NO2(GT)|PT08.S4(NO2)|PT08.S5(O3)|   T| RH| AH|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+---+---+
|15/03/2020|14.00.00|   1.2|       1300|      50|     0.3|         1100|     80|        1000|     40|        1500|       1200|26.5| 45|0.8|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+---+---+



# DELETE

In [87]:
spark.sql("""
DELETE FROM local.sample_data
WHERE Date = '15/03/2020' 
""")


DataFrame[]

In [89]:
spark.sql("SELECT * from local.sample_data WHERE date = '15/03/2020'").show()

+----+----+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+---+---+---+
|Date|Time|CO(GT)|PT08.S1(CO)|NMHC(GT)|C6H6(GT)|PT08.S2(NMHC)|NOx(GT)|PT08.S3(NOx)|NO2(GT)|PT08.S4(NO2)|PT08.S5(O3)|  T| RH| AH|
+----+----+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+---+---+---+
+----+----+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+---+---+---+

